In [69]:
import pandas as pd

In [130]:
raw_df = pd.read_csv('masterData-2023-09-04.csv', dtype={'label':'category'})
# raw_df.head()

In [131]:
df = raw_df.drop(columns=['adjclose'])
df['range'] = df['high'] - df['low']
df['value_type'] = 'nominal'
ratios = {'10': 3.618, '9': 2.618, '8': 1.618, '7': 1.382, '6':1.118, '5':1, '76.2':.764, '4': .618, '50': .5, '3': .382, '2': .236, '1':.118}
curves = ['RC', 'FC']
columns = {'JxD': ['high', -0.5], 'BDP': ['low', 0.382], 'WDP': ['high', -0.382]}
for ratio in ratios.keys():
    columns[f"RC{ratio}"] = ['close', ratios[ratio]]
for ratio in reversed(ratios.keys()):
    columns[f"FC{ratio}"] = ['close', -ratios[ratio]]

In [125]:
for column in columns.keys():
    df[column] = df[columns[column][0]]+df['range']*columns[column][1]

bound_multiplier = 0.073

df_lb = df.copy()
df_lb[list(columns.keys())] = df_lb[list(columns.keys())].sub(df_lb['range']*bound_multiplier, axis='index')
df_lb['value_type'] = 'lower'

df_ub = df.copy()
df_ub[list(columns.keys())] = df_ub[list(columns.keys())].add(df_ub['range']*bound_multiplier, axis='index')
df_ub['value_type'] = 'upper'
# df.head()

df_consolidated = pd.concat([df_lb, df, df_ub])
df_consolidated.sort_index(level=['symbol','label','value_type'], inplace=True)
df_consolidated.set_index(['symbol','label','date', 'open','high','low',
                           'close','range','value_type'], inplace=True)

In [132]:
df_consolidated.to_excel('stocksAnalyzed-23-09-04.xlsx')